In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("Delivery_Logistics.csv")

In [3]:
df.shape

(1200, 15)

In [4]:
df.head()

,Date,Region,Category,Supplier,Warehouse,Order Status,Units Sold,Inventory Level,Transportation Cost,Order Accuracy,Lead Time (Days),Backorder,Cost of Goods Sold (COGS),Average Inventory,Warehouse Capacity
0,14-01-2020,North,Accessories,Supplier A,Warehouse 1,Fulfilled,302,2124,1103.838324,True,9,False,37820.05240,2048.0,5037
1,15-11-2020,East,Furniture,Supplier D,Warehouse 2,Fulfilled,741,1972,13163.007660,True,11,False,54396.17369,1213.0,9216
2,16-04-2020,East,Furniture,Supplier C,Warehouse 2,Fulfilled,940,454,9872.294126,True,17,False,24217.45462,1160.5,7699
3,17-04-2020,South,Accessories,Supplier D,Warehouse 2,Canceled,589,1867,4547.589932,True,3,False,38919.08005,3364.5,9271
4,18-04-2020,North,Accessories,Supplier C,Warehouse 2,Fulfilled,964,4862,11994.042310,True,21,False,59204.50683,3063.0,5828


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Date                       1200 non-null   object 
 1   Region                     1200 non-null   object 
 2   Category                   1200 non-null   object 
 3   Supplier                   1200 non-null   object 
 4   Warehouse                  1200 non-null   object 
 5   Order Status               1200 non-null   object 
 6   Units Sold                 1200 non-null   int64  
 7   Inventory Level            1200 non-null   int64  
 8   Transportation Cost        1200 non-null   float64
 9   Order Accuracy             1200 non-null   bool   
 10  Lead Time (Days)           1200 non-null   int64  
 11  Backorder                  1200 non-null   bool   
 12  Cost of Goods Sold (COGS)  1200 non-null   float64
 13  Average Inventory          1200 non-null   float

In [6]:
df.describe()

,Units Sold,Inventory Level,Transportation Cost,Lead Time (Days),Cost of Goods Sold (COGS),Average Inventory,Warehouse Capacity
count,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000
mean,533.892500,2530.473333,7723.518494,15.739167,59370.480017,2529.588333,7425.831667
std,275.469524,1416.945265,4202.727652,8.442187,44542.532052,1000.512796,1460.827133
min,50.000000,101.000000,509.098991,1.000000,1415.613541,235.000000,5002.000000
25%,298.000000,1261.750000,4145.882050,8.000000,22982.367025,1843.000000,6116.500000
50%,529.000000,2623.500000,7759.647979,16.000000,47773.376750,2496.500000,7427.000000
75%,777.000000,3726.250000,11154.018832,23.000000,88006.544317,3240.125000,8662.250000
max,999.000000,4999.000000,14997.814050,29.000000,190841.495400,4863.500000,9995.000000


In [8]:
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

In [10]:
df['inventory_status'] = np.where(
    df['Inventory Level'] < df['Average Inventory'], 'Understock',
    np.where(df['Inventory Level'] > df['Average Inventory'], 'Overstock', 'Optimal')
)

In [11]:
df['inventory_turnover'] = df['Units Sold'] / df['Average Inventory']

In [12]:
df['lead_time_category'] = pd.cut(
    df['Lead Time (Days)'],
    bins=[0, 7, 14, 30],
    labels=['Fast', 'Medium', 'Slow']
)

In [13]:
df['cost_per_unit'] = df['Transportation Cost'] / df['Units Sold']

In [14]:
df['warehouse_utilization'] = df['Average Inventory'] / df['Warehouse Capacity']

In [15]:
df[['inventory_status', 'inventory_turnover', 'lead_time_category',
    'cost_per_unit', 'warehouse_utilization']].head()

,inventory_status,inventory_turnover,lead_time_category,cost_per_unit,warehouse_utilization
0,Overstock,0.147461,Medium,3.655094,0.406591
1,Overstock,0.610882,Medium,17.763843,0.131619
2,Understock,0.809996,Slow,10.502441,0.150734
3,Understock,0.175063,Fast,7.720866,0.362906
4,Overstock,0.314724,Slow,12.441953,0.525566


In [16]:
df['inventory_status'].value_counts(normalize=True) * 100

inventory_status
Overstock     50.25
Understock    49.75
Name: proportion, dtype: float64

In [17]:
pd.crosstab(df['Category'], df['inventory_status'], normalize='index') * 100

inventory_status,Overstock,Understock
Category,,
Accessories,53.903346,46.096654
Clothing,49.517685,50.482315
Electronics,52.547771,47.452229
Furniture,45.424837,54.575163


In [18]:
df[['Category', 'inventory_turnover']] \
    .groupby('Category') \
    .mean() \
    .sort_values(by='inventory_turnover')

,inventory_turnover
Category,
Clothing,0.258933
Accessories,0.268843
Electronics,0.288451
Furniture,0.298436


In [19]:
df['warehouse_utilization'].describe()

count    1200.000000
mean        0.354684
std         0.159285
min         0.025984
25%         0.242709
50%         0.341771
75%         0.452807
max         0.930221
Name: warehouse_utilization, dtype: float64

In [20]:
df['warehouse_risk'] = np.where(
    df['warehouse_utilization'] > 0.85, 'Over Utilized',
    np.where(df['warehouse_utilization'] < 0.4, 'Under Utilized', 'Optimal')
)

In [21]:
df['warehouse_risk'].value_counts()

warehouse_risk
Under Utilized    755
Optimal           443
Over Utilized       2
Name: count, dtype: int64

In [23]:
df['backorder_flag'] = df['Backorder'].astype(int)

In [24]:
df['backorder_flag'].head()

0    0
1    0
2    0
3    0
4    0
Name: backorder_flag, dtype: int64

In [25]:
df['backorder_flag'].mean() * 100

np.float64(9.666666666666666)

In [26]:
df.groupby('Supplier')['backorder_flag'] \
  .mean() \
  .sort_values(ascending=False) * 100

Supplier
Supplier D    12.211221
Supplier B    10.322581
Supplier C     9.090909
Supplier A     7.051282
Name: backorder_flag, dtype: float64

In [27]:
df.groupby('lead_time_category')['backorder_flag'].mean() * 100

C:\Users\dk523\AppData\Local\Temp\ipykernel_18972\3721080189.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('lead_time_category')['backorder_flag'].mean() * 100


lead_time_category
Fast       8.955224
Medium    10.000000
Slow       9.821429
Name: backorder_flag, dtype: float64

In [28]:
df['Transportation Cost'].sum()

np.float64(9268222.192554599)

In [29]:
df.groupby('Category')['Transportation Cost'].sum() \
  .sort_values(ascending=False)

Category
Electronics    2.520407e+06
Furniture      2.447170e+06
Clothing       2.320770e+06
Accessories    1.979876e+06
Name: Transportation Cost, dtype: float64

In [30]:
df.groupby('backorder_flag')['Transportation Cost'].mean()

backorder_flag
0    7757.491122
1    7406.050136
Name: Transportation Cost, dtype: float64

In [31]:
df[['Inventory Level', 'Transportation Cost']].corr()

,Inventory Level,Transportation Cost
Inventory Level,1.000000,-0.028524
Transportation Cost,-0.028524,1.000000


In [32]:
df.to_csv("supply_chain_cleaned.csv", index=False)